In [814]:
import warnings

import re
import pandas as pd
import numpy as np
import seaborn as sns
import nltk

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

warnings.filterwarnings("ignore")

In [815]:
#ЧЕКПОИНТ РЕШЕНИЯ
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv("submission.csv")

nltk.download('stopwords')

#ПРЕДОБРАБОТКА ТЕКСТА
train = train.rename(columns={'Решение             ': "Решение"})
test = test.rename(columns={'Решение             ': "Решение"})

patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    return tokens

text_content = train['Содержание'].astype(str).str.lower().apply(lemmatize)
text_solution = train['Решение'].astype(str).str.lower().apply(lemmatize)
for i in range(text_content.shape[0]):
    text_content[i] = " ".join(text_content[i])
    text_solution[i] = " ".join(text_solution[i])
    
train.loc[:, "Содержание"] = text_content
train.loc[:, "Решение"] = text_solution
    
text_content = test['Содержание'].astype(str).str.lower().apply(lemmatize)
text_solution = test['Решение'].astype(str).str.lower().apply(lemmatize)
for i in range(text_content.shape[0]):
    text_content[i] = " ".join(text_content[i])
    text_solution[i] = " ".join(text_solution[i])

test.loc[:, "Содержание"] = text_content
test.loc[:, "Решение"] = text_solution

#НЕНУЖНЫЕ В БАН
train = train.drop(["Дата закрытия обращения", "Место"], axis=1)
test = test.drop(["Дата закрытия обращения", "Место"], axis=1)

#НОВЫЕ ГЕНЕРИРУЕМ
train.loc[:, "Дедлайн пропущен"] = pd.to_datetime(train["Дата восстановления"]) - pd.to_datetime(train["Крайний срок"])
train.loc[:, "Дедлайн пропущен"] = train["Дедлайн пропущен"].dt.days > 0
test.loc[:, "Дедлайн пропущен"] = pd.to_datetime(test["Дата восстановления"]) - pd.to_datetime(test["Крайний срок"])
test.loc[:, "Дедлайн пропущен"] = test["Дедлайн пропущен"].dt.days > 0

train.loc[:, "Время выполнения"] = pd.to_datetime(train["Дата восстановления"]) - pd.to_datetime(train["Дата обращения"])
test.loc[:, "Время выполнения"] = pd.to_datetime(test["Дата восстановления"]) - pd.to_datetime(test["Дата обращения"])

def date_std(array):
    return np.sqrt(np.sum((array - np.mean(array)) ** 2) / len(array))

data = train.groupby(by=["Критичность", "Влияние"])["Время выполнения"].aggregate(np.mean)
values_dict = {}

for i in range(len(data)):
    values_dict[data.index[i]] = data[i]

new_col = []
for row in range(len(train)):
    new_col.append(train.iloc[row, -1] - values_dict[(train.iloc[row, 12], train.iloc[row, 13])])
    
train.loc[:, "Кр./влияние"] = new_col
train.loc[:, "Кр./влияние"] = train['Кр./влияние'].astype("timedelta64[s]")

datа = test.groupby(by=["Критичность", "Влияние"])["Время выполнения"].aggregate(np.mean)
values_dict = {}

for i in range(len(data)):
    values_dict[data.index[i]] = data[i]

new_col = []
for row in range(len(test)):
    new_col.append(test.iloc[row, -1] - values_dict[(test.iloc[row, -5], test.iloc[row, -4])])
    
test.loc[:, "Кр./влияние"] = new_col
test.loc[:, "Кр./влияние"] = test['Кр./влияние'].astype("timedelta64[s]")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxxx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [817]:
test.head()

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Критичность,Влияние,Система,Дедлайн пропущен,Время выполнения,Кр./влияние
0,14803,заявка предоставление отзыв право доступ ресурс,Сервис41,3-Низкий,Закрыт,ФГ13,2018-02-22 07:51,2018-02-16 13:11,Запрос,NaN,работа обращение выполнить,NaN,2018-02-18 04:32,4-Нет,4-Нет влияния,Система43,False,1 days 15:21:00,-79191.0
1,14804,транспортный шлюз,Сервис61,2-Средний,Закрыт,ФГ1,2018-02-21 14:36,2018-02-16 13:13,Запрос,NaN,проинф,NaN,2018-02-16 17:55,1-Особая,3-Малое,Система61,False,0 days 04:42:00,-50717.0
2,14805,заявка предоставление отзыв право доступ ресурс,Сервис22,3-Низкий,Закрыт,ФГ13,2018-03-31 06:36,2018-02-16 13:13,Запрос,NaN,работа обращение выполнить,NaN,2018-03-15 16:09,2-Повышенная,4-Нет влияния,Система13,False,27 days 02:56:00,2187290.0
3,14806,заявка предоставление отзыв право доступ ресурс,Сервис91,3-Низкий,Отменен,ФГ13,2018-02-21 14:29,2018-02-16 13:14,Запрос,NaN,запрос отменить пользователь портал самообслуж...,NaN,2018-02-16 13:15,2-Повышенная,4-Нет влияния,Система363,False,0 days 00:01:00,-156010.0
4,14807,заявка предоставление отзыв право доступ ресурс,Сервис22,3-Низкий,Закрыт,ФГ13,2018-03-30 09:36,2018-02-16 13:14,Запрос,NaN,работа обращение выполнить,NaN,2018-03-15 16:09,2-Повышенная,4-Нет влияния,Система13,False,27 days 02:55:00,2187230.0


In [1024]:
# ДЛЯ АНОМАЛИЙ

total_dataset = pd.DataFrame()
#не характерная длительность решения по приоритету
#print(len(total_dataset), end='->')
#total_dataset = train[train["Приоритет"] == "0-Критический"].copy()
#total_dataset = pd.concat([total_dataset, train[(train["Приоритет"] == "1-Высокий") & (train["Время выполнения"].dt.total_seconds() / (60 * 60) > 2)]], axis=0)
#total_dataset = pd.concat([total_dataset, train[(train["Приоритет"] == "2-Средний") & (train["Время выполнения"].dt.total_seconds() / (60 * 60) > 4)]], axis=0)
#total_dataset = pd.concat([total_dataset, train[(train["Приоритет"] == "3-Низкий") & (train["Время выполнения"].dt.total_seconds() / (60 * 60) > 8)]], axis=0)
#total_dataset = pd.concat([total_dataset, train[train["Время выполнения"].isna()]], axis=0)
#print(len(total_dataset))

#test[test["Приоритет"] == "0-Критический"] #0
#test[(test["Приоритет"] == "1-Высокий") & (test["Время выполнения"].dt.total_seconds() / (60 * 60) > 2)] #0
#test[(test["Приоритет"] == "2-Средний") & (test["Время выполнения"].dt.total_seconds() / (60 * 60) > 4)]
#test[(test["Приоритет"] == "3-Низкий") & (test["Время выполнения"].dt.total_seconds() / (60 * 60) > 8)]
#test[test["Время выполнения"].isna()]

#невозможные сроки выполнения
print(len(total_dataset), end='->')
total_dataset = pd.concat([total_dataset, train[train["Время выполнения"].dt.total_seconds() < 0]], axis=0)
test[test["Время выполнения"].dt.total_seconds() < 0] #0
print(len(total_dataset))

#неправильная классификация (был запросом, на самом деле инцидент), либо сразу был инцидентом НАДО ЕЩЁ ДЛЯ ТЕСТА ТО ЖЕ САМОЕ
print(len(total_dataset), end='->')
data_only12 = train[(train["Тип переклассификации"] > 0)] #запрос->инцидент=1
data_0_inc = train[(train["Тип переклассификации"] == 0) & (train["Тип обращения итоговый"] == "Инцидент")]

#сборник плохих значений признака (аномалия)
total_result = {}
#колонки, по которым проверяемся
main_cols = ["Содержание", "Сервис", "Статус", "Функциональная группа", "Решение", "Система"]

#получаем словарь, где ключ - наименование колонки, значение - лист тех значений, которые позволяют записи быть аномальной
for col in main_cols:
    subset = pd.DataFrame(data_only12.groupby(by=col))
    result={}
    col_index = list(train.columns).index(col)
    for i in range(len(subset)):
        data = subset.iloc[i, 1]
        result[data.iloc[0, col_index]] = (len(data[data["Тип переклассификации"] == 1]), len(data[data["Тип переклассификации"] == 2]))
    total_result[col] = [name for name in result.keys() if result[name][0] > 0 and result[name][1] == 0]
    
    #маска для проверки принадлежности записи к аномальной по значению колонки
    mask = []
    cur_data = data_only12.copy()
    for i in range(len(data_only12)):
        mask.append(data_only12.iloc[i, col_index] in total_result[col])
    cur_data.loc[:, "mask"] = mask
    cur_data = cur_data[cur_data["mask"] == True]#.drop("mask", axis=1)
    
    #подсоединение в общий датасет
    if len(total_dataset) == 0:
        total_dataset = cur_data.copy()
    else:
        total_dataset = pd.concat([total_dataset, cur_data], axis=0).reset_index().iloc[:, 1:]
total_dataset = total_dataset.drop('mask', axis=1)
total_dataset = pd.concat([total_dataset, data_0_inc], axis=0).reset_index().iloc[:, 1:]
print(len(total_dataset))

#аномальные запросы по системе/функциональной группе
print(len(total_dataset), end='->')
main_cols = ["Система", "Функциональная группа"]
for col in main_cols:
    data = pd.DataFrame(train.groupby(by=col))
    for row in range(len(data)):
        subset = data.iloc[row, 1]
        mask = subset["Содержание"].value_counts().reset_index().iloc[:, 1] < np.percentile(subset["Содержание"].value_counts().reset_index().iloc[:, 1].unique(), q=5)
        subset.loc[:, 'mask'] = mask
        subset = subset[subset['mask'] == True].drop('mask', axis=1)
        total_dataset = pd.concat([total_dataset, subset], axis=0)
print(len(total_dataset))

#причинно-следственные связи (проблема в банке / проблемы извне)

#уникальные решения (уникальные содержания?)

#содержание - решение

0->1
1->630
630->730


In [1025]:
len(total_dataset.drop_duplicates())

446